In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

import pipelines
from s3_client import s3_client
import columns as c
import common

pd.set_option('float_format', '{:f}'.format)

s3_client = s3_client() 

credentials.py 16:51:01 INFO: Found credentials in shared credentials file: ~/.aws/credentials


In [2]:
_df = s3_client.read_newest_df_from_s3(common.FINAL_DATA_PATH.format(data_type='rent'))

s3_client.py 16:51:03 INFO: Downloading rent/final/rent_final_2019_12_28T00_22_22.parquet from flats-data bucket ...
s3_client.py 16:51:06 INFO: Successfully downloaded rent/final/rent_final_2019_12_28T00_22_22.parquet from flats-data bucket.


In [3]:
df = _df.drop(c.HEATING, axis=1)
print(df.shape)
df = df.dropna()
print(df.shape)
df[c.DATE_ADDED] = pd.to_datetime(df[c.DATE_ADDED]).dt.strftime("%Y%m%d").astype(int)
df[c.DATE_REFRESHED] = pd.to_datetime(df[c.DATE_REFRESHED]).dt.strftime("%Y%m%d").astype(int)

(85447, 41)
(81779, 41)


In [4]:
X = df.drop([c.PRICE, c.PRICE_M2, c.OFFER_ID], axis=1)
y = df[c.PRICE_M2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_depth = [int(x) for x in np.linspace(4, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]

random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

rf = RandomForestRegressor()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(
    estimator = rf,
    param_distributions = random_grid,
    n_iter = 10,
    cv = 3,
    verbose=3,
    random_state=42,
    n_jobs = -1,
)

# Fit the random search model
rf_random.fit(X_train, y_train)

def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = abs(predictions - y_test)
    mape = 100 * np.mean(errors / y_test)
    print('Model Performance')
    print('Average Error: {:0.4f}'.format(np.mean(errors)))
    print('MAPE = {:0.2f}%.'.format(mape))
    return mape

# Fit the grid search to the data
best_rf = rf_random.best_estimator_

grid_accuracy = evaluate(best_rf, X_test, y_test)

feature_importances = (pd.DataFrame(best_rf.feature_importances_,
                                    index = X_train.columns,
                                    columns=['importance'])
                       .sort_values('importance', ascending=False))
feature_importances

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
